<a href="https://colab.research.google.com/github/afeld/python-public-policy/blob/master/lecture_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NYU Wagner - Python Coding for Public Policy**
# Class 1: Data cleanup, iterating with "for" loops, basic summary stats

# LECTURE

## Start by importing necessary packages

In [1]:
!pip install --quiet gdown
import pandas as pd
import gdown

In [18]:
# You can use pd.set_option() to make sure you see all the rows in your dataframe
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', None)  # or 199

## Read and save 311 Service Requests dataset as a pandas dataframe

In [ ]:
drive.mount('/content/drive')

import gdown
gdown.cached_download("https://drive.google.com/uc?id=1woUPU62mGXRtVvZmpzAwmrYl01vLK4PJ", "311_Service_Requests_2018-19.csv")

In [3]:
df = pd.read_csv('311_Service_Requests_2018-19.csv')

/Users/afeld/.asdf/installs/python/3.8.0/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (8,17,20,31,34) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## **Today's goal**: learn which 311 complaints are most common and which agencies are responsible for handling them. But first, data clean up!

## Excluding bad records from the dataframe

First, let's refresh ourselves on what the invalid complaint types are, by getting the distinct list of all complaint types

In [19]:
# the set() function returns all unique values in a column
set(df['Complaint Type'])

{'"-->\'-->`-->&...',
 '$(sleep 11)',
 '${3804*3137}',
 '%2e%2e%2f%2e%2e%2f%2e%2e%2f...',
 '%2e%2e%5c%2e%2e%5c%2e%2e%5c...',
 '%2fetc%2fpasswd',
 '%E5%98%8A%E5%98%8DX-Injecti...',
 '%c0%ae/%c0%ae/%c0%ae/%c0%ae...',
 '%c0%ae/%c0%ae/%c0%ae/WEB-IN...',
 '%c0%ae/%c0%ae/WEB-INF/web.xml',
 '%c0%ae/WEB-INF/web.xml',
 '%{(#dm=@ognl.OgnlContext@DE...',
 '%{4761*8506}',
 '() { :;}; /bin/sleep 0',
 '() { :;}; /bin/sleep 11',
 '() { _; } >_',
 '(select extractvalue(xmltyp...',
 "(select load_file('\\\\\\\\615h...",
 '*)(!(objectClass=*)',
 '*)(objectClass=*',
 '.../....///.../....///.../....',
 '.../...//.../...//.../...//...',
 '..././..././..././..././......',
 '.../.\\.../.\\.../.\\.../.\\......',
 '.../Misc. Comments',
 '...\\./...\\./...\\./...\\./......',
 '...\\.\\...\\.\\...\\.\\...\\.\\......',
 '../../../../../../../../../...',
 '../../../../WEB-INF/web.xml',
 '../../../WEB-INF/web.xml',
 '../../../WEB-INF/web.xml;x=',
 '../../WEB-INF/web.xml',
 '../../WEB-INF/web.xml;x=',
 '../WEB-INF/w

Let's see how frequently these invalid Complaint Type values appear in the data.

Use `.groupby().size()` to get the count of 311 requests per complaint type value. This is very similar to a pivot table in Excel.

In [20]:
# remember .size gives you the count of cells across all columns in the dataframe
df.size

117227200

In [21]:
# to just get the total count of records in the dataset, we should get the size of the 'Unique Key' column
df['Unique Key'].size

2859200

In [22]:
# watch out! the .groupby().size() function doesn't work that same way. it gets the count of number of rows in each group

df.groupby('Complaint Type').size()

# let's break down what this code is doing:
# first you need to group the records in the dataset based on their 'Complaint Type' value
# then you can count the records that have been grouped together by their shared 'Complaint Type" value

Complaint Type
"-->'-->`-->&...                                  1
$(sleep 11)                                       1
${3804*3137}                                      1
%2e%2e%2f%2e%2e%2f%2e%2e%2f...                    1
%2e%2e%5c%2e%2e%5c%2e%2e%5c...                    1
%2fetc%2fpasswd                                   1
%E5%98%8A%E5%98%8DX-Injecti...                    1
%c0%ae/%c0%ae/%c0%ae/%c0%ae...                    1
%c0%ae/%c0%ae/%c0%ae/WEB-IN...                    1
%c0%ae/%c0%ae/WEB-INF/web.xml                     1
%c0%ae/WEB-INF/web.xml                            1
%{(#dm=@ognl.OgnlContext@DE...                    1
%{4761*8506}                                      1
() { :;}; /bin/sleep 0                            1
() { :;}; /bin/sleep 11                           1
() { _; } >_                                      1
(select extractvalue(xmltyp...                    1
(select load_file('\\\\615h...                    1
*)(!(objectClass=*)                              

It looks like most invalid complaint types only have a few records. Try excluding all complaint type categories with < 4 records, assuming that all complaint type categories with < 4 instances in the data are bad data entries.

Why 4? It's arbitrary. We're looking for trends in the data in this case don't care about low frequency entries.

Create a dataframe that captures the count of records per `Complaint Type` value.

In [23]:
counts = df.groupby('Complaint Type').size().reset_index(name='count')
counts

# .reset_index(name='count') allows us to name the new column that contains the count of rows

,Complaint Type,count
0,"""-->'-->`-->&...",1
1,$(sleep 11),1
2,${3804*3137},1
3,%2e%2e%2f%2e%2e%2f%2e%2e%2f...,1
4,%2e%2e%5c%2e%2e%5c%2e%2e%5c...,1
5,%2fetc%2fpasswd,1
6,%E5%98%8A%E5%98%8DX-Injecti...,1
7,%c0%ae/%c0%ae/%c0%ae/%c0%ae...,1
8,%c0%ae/%c0%ae/%c0%ae/WEB-IN...,1
9,%c0%ae/%c0%ae/WEB-INF/web.xml,1


You can also use `.count()` but [the output is a little different](https://stackoverflow.com/questions/33346591/what-is-the-difference-between-size-and-count-in-pandas).

Create a "series" that only lists the `Complaint Type` values that have record counts > 4. (Remember: A single column from a pandas dataframe is called a series. It's essentially a list containing all the values in the column.) 

In [26]:
valid_complaint_types = counts['Complaint Type'][counts['count'] > 4]
valid_complaint_types

45                                     APPLIANCE
46                             Abandoned Vehicle
48                               Advocate - Lien
49                              Advocate - Other
50                               Advocate - RPIE
52                 Advocate-Co-opCondo Abatement
53                Advocate-Commercial Exemptions
55                  Advocate-Personal Exemptions
57                 Advocate-Prop Refunds/Credits
58                       Advocate-Property Value
60                                   Air Quality
61                              Alzheimer's Care
62                                  Animal Abuse
63                   Animal Facility - No Permit
64                              Animal in a Park
65                                  Animal-Abuse
67                                      Asbestos
68                              BEST/Site Safety
69                    Beach/Pool/Sauna Complaint
70                      Benefit Card Replacement
71                  

Filter our `df` dataframe to only keep the rows where the `Complaint Type` value is in the `valid_complaint_types` series we created in the previous step. Save the result in a new dataframe.

In [32]:
df_cleaned = df[df['Complaint Type'].isin(valid_complaint_types)]

How can we make sure this worked? Let's check how many records there were originally in `df` vs how many are in `df_cleaned`.

Before:

In [33]:
df['Unique Key'].size

2859200

After:

In [34]:
df_cleaned['Unique Key'].size

2859011

We can also print the set of complaint_type values from our cleaned dataframe to make sure they look correct.

In [35]:
df_cleaned['Complaint Type']

0                   Homeless Person Assistance
1                                  DOOR/WINDOW
2                          Noise - Residential
3                                        Noise
4          Request Large Bulky Item Collection
                          ...                 
2859195                Noise - Street/Sidewalk
2859196                Noise - Street/Sidewalk
2859197                        Noise - Vehicle
2859198                Noise - Street/Sidewalk
2859199                        Noise - Vehicle
Name: Complaint Type, Length: 2859011, dtype: object

Now we can overwrite `df` with `df_cleaned`.

In [36]:
df = df_cleaned

Great, now those invalid records will be excluded from our analysis!

Another approach to excluding those invalid records would be to use ["regex" (regular expressions)](https://www.w3schools.com/python/python_regex.asp) to find records with weird characters.

## Column cleanup

We should also improve our data by making columns easier to reference. This requires a "for" loop.

### Introduction to "for" loops for iterating through items in a list

In [37]:
list_of_numbers = [1, 3, 5, 7, 9]
for number in list_of_numbers:
  print(number)

1
3
5
7
9


In [38]:
for number in list_of_numbers:
  print(number + 4)

5
7
9
11
13


You can also iterate through characters in a string.

In [39]:
practice_text = 'Hello my name is Aidan'
for letter in practice_text:
  print(letter)

H
e
l
l
o
 
m
y
 
n
a
m
e
 
i
s
 
A
i
d
a
n


Characters can be accessed by their index in the string.

In [40]:
practice_text[6]

'm'

In [41]:
# using the len() function gives you the length (count of charaters) of a string

len(practice_text)

22

In [42]:
n = 0
for letter in practice_text:
  print(letter, n)
  n += 1 # same as n = n + 1

H 0
e 1
l 2
l 3
o 4
  5
m 6
y 7
  8
n 9
a 10
m 11
e 12
  13
i 14
s 15
  16
A 17
i 18
d 19
a 20
n 21


In [43]:
# you can also replace characters in a string using the .replace() function

practice_text.replace(' ', '_')

'Hello_my_name_is_Aidan'

## How to use "for" loops to iterate through dataframes 
### Let's print a sentence explaining each 311 complaint using a "for" loop


In [46]:
# first let's create a smaller dataframe containing a subset of the 311 data. We don't want to loop through all 2.8 million rows for this learning exercise

first_3 = df.head(3)

In [47]:
# to iterate through a dataframe, you need to use a function called iterrows() that finds each index and corresponding row in a dataframe

for myindex, row in first_3.iterrows():
    print(myindex)
    print(row)

0
Unique Key                                                                                                                                   39888071
Created Date                                                                                                                   08/01/2018 12:00:10 AM
Closed Date                                                                                                                    08/01/2018 01:52:46 AM
Agency                                                                                                                                            DHS
Agency Name                                                                                         Operations Unit - Department of Homeless Services
Complaint Type                                                                                                             Homeless Person Assistance
Descriptor                                                                                        

In [48]:
# we can access data from the row using the column name

for index, row in first_3.iterrows():
     print(index, 'On', row['Created Date'], ', a user filed a request to', row['Agency'], 'for', row['Complaint Type'])

0 On 08/01/2018 12:00:10 AM , a user filed a request to DHS for Homeless Person Assistance
1 On 08/01/2018 12:00:26 AM , a user filed a request to HPD for DOOR/WINDOW
2 On 08/01/2018 12:00:54 AM , a user filed a request to NYPD for Noise - Residential


In [49]:
# instead of saving the first N rows of the dataframe (df) as a new object, we could have also done this

for index, row in df.head(20).iterrows():
     print(index, 'On', row['Created Date'], ', a user filed a request to', row['Agency'], 'for', row['Complaint Type'])

0 On 08/01/2018 12:00:10 AM , a user filed a request to DHS for Homeless Person Assistance
1 On 08/01/2018 12:00:26 AM , a user filed a request to HPD for DOOR/WINDOW
2 On 08/01/2018 12:00:54 AM , a user filed a request to NYPD for Noise - Residential
3 On 08/01/2018 12:01:00 AM , a user filed a request to DEP for Noise
4 On 08/01/2018 12:01:00 AM , a user filed a request to DSNY for Request Large Bulky Item Collection
5 On 08/01/2018 12:01:09 AM , a user filed a request to NYPD for Noise - Street/Sidewalk
6 On 08/01/2018 12:01:10 AM , a user filed a request to NYPD for Noise - Street/Sidewalk
7 On 08/01/2018 12:01:58 AM , a user filed a request to NYPD for Noise - Commercial
8 On 08/01/2018 12:03:13 AM , a user filed a request to NYPD for Blocked Driveway
9 On 08/01/2018 12:03:54 AM , a user filed a request to NYPD for Noise - Street/Sidewalk
10 On 08/01/2018 12:03:55 AM , a user filed a request to NYPD for Noise - Street/Sidewalk
11 On 08/01/2018 12:04:41 AM , a user filed a request 

## Using a "for" loop to improve our column names

There are two different ways to access a column in a dataframe: `df['column_name']` and `df.column_name`, but the latter only works if the header doesn't have spaces in it.

In [50]:
# get list of all columnn names. many of these have spaces in them which makes it harder to reference in the code.

df.columns

Index(['Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Agency Name',
       'Complaint Type', 'Descriptor', 'Location Type', 'Incident Zip',
       'Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2',
       'Intersection Street 1', 'Intersection Street 2', 'Address Type',
       'City', 'Landmark', 'Facility Type', 'Status', 'Due Date',
       'Resolution Description', 'Resolution Action Updated Date',
       'Community Board', 'BBL', 'Borough', 'X Coordinate (State Plane)',
       'Y Coordinate (State Plane)', 'Open Data Channel Type',
       'Park Facility Name', 'Park Borough', 'Vehicle Type',
       'Taxi Company Borough', 'Taxi Pick Up Location', 'Bridge Highway Name',
       'Bridge Highway Direction', 'Road Ramp', 'Bridge Highway Segment',
       'Latitude', 'Longitude', 'Location'],
      dtype='object')

In [51]:
# we can use a "for" loop to rename the columns and get rid of the spaces and make the names lowercase

for column_name in df.columns:
  df.rename(columns={ column_name: column_name.replace(' ', '_').lower() }, inplace=True)

df.columns

Index(['unique_key', 'created_date', 'closed_date', 'agency', 'agency_name',
       'complaint_type', 'descriptor', 'location_type', 'incident_zip',
       'incident_address', 'street_name', 'cross_street_1', 'cross_street_2',
       'intersection_street_1', 'intersection_street_2', 'address_type',
       'city', 'landmark', 'facility_type', 'status', 'due_date',
       'resolution_description', 'resolution_action_updated_date',
       'community_board', 'bbl', 'borough', 'x_coordinate_(state_plane)',
       'y_coordinate_(state_plane)', 'open_data_channel_type',
       'park_facility_name', 'park_borough', 'vehicle_type',
       'taxi_company_borough', 'taxi_pick_up_location', 'bridge_highway_name',
       'bridge_highway_direction', 'road_ramp', 'bridge_highway_segment',
       'latitude', 'longitude', 'location'],
      dtype='object')

In [53]:
# we could accomplish that more concisely by setting df.columns = a list we create using an in-line for loop

# list produced using an in-line for loop
print([c.replace(' ', '_').lower() for c in df.columns])

['unique_key', 'created_date', 'closed_date', 'agency', 'agency_name', 'complaint_type', 'descriptor', 'location_type', 'incident_zip', 'incident_address', 'street_name', 'cross_street_1', 'cross_street_2', 'intersection_street_1', 'intersection_street_2', 'address_type', 'city', 'landmark', 'facility_type', 'status', 'due_date', 'resolution_description', 'resolution_action_updated_date', 'community_board', 'bbl', 'borough', 'x_coordinate_(state_plane)', 'y_coordinate_(state_plane)', 'open_data_channel_type', 'park_facility_name', 'park_borough', 'vehicle_type', 'taxi_company_borough', 'taxi_pick_up_location', 'bridge_highway_name', 'bridge_highway_direction', 'road_ramp', 'bridge_highway_segment', 'latitude', 'longitude', 'location']


In [55]:
# set df.columns equal to the list
df.columns = [column.replace(' ', '_').lower() for column in df.columns]
df.columns

Index(['unique_key', 'created_date', 'closed_date', 'agency', 'agency_name',
       'complaint_type', 'descriptor', 'location_type', 'incident_zip',
       'incident_address', 'street_name', 'cross_street_1', 'cross_street_2',
       'intersection_street_1', 'intersection_street_2', 'address_type',
       'city', 'landmark', 'facility_type', 'status', 'due_date',
       'resolution_description', 'resolution_action_updated_date',
       'community_board', 'bbl', 'borough', 'x_coordinate_(state_plane)',
       'y_coordinate_(state_plane)', 'open_data_channel_type',
       'park_facility_name', 'park_borough', 'vehicle_type',
       'taxi_company_borough', 'taxi_pick_up_location', 'bridge_highway_name',
       'bridge_highway_direction', 'road_ramp', 'bridge_highway_segment',
       'latitude', 'longitude', 'location'],
      dtype='object')

More info on single line for loops: https://blog.teamtreehouse.com/python-single-line-loops

In [56]:
# now we can reference the complaint_type column simply using a .

df.complaint_type

0                   Homeless Person Assistance
1                                  DOOR/WINDOW
2                          Noise - Residential
3                                        Noise
4          Request Large Bulky Item Collection
                          ...                 
2859195                Noise - Street/Sidewalk
2859196                Noise - Street/Sidewalk
2859197                        Noise - Vehicle
2859198                Noise - Street/Sidewalk
2859199                        Noise - Vehicle
Name: complaint_type, Length: 2859011, dtype: object

See how this revised `for` loop is cleaner to understand now?

**Before:**

```python
for index, row in df.head().iterrows():
     print(index, 'On', row['Created Date'], ', a user filed a request to', row['Agency'], 'for', row['Complaint Type'])
```

**After:**

In [60]:
for index, row in df.head().iterrows():
     print(index, 'On', row.created_date, ', a user filed a request to', row.agency, 'for', row.complaint_type)

0 On 08/01/2018 12:00:10 AM , a user filed a request to DHS for Homeless Person Assistance
1 On 08/01/2018 12:00:26 AM , a user filed a request to HPD for DOOR/WINDOW
2 On 08/01/2018 12:00:54 AM , a user filed a request to NYPD for Noise - Residential
3 On 08/01/2018 12:01:00 AM , a user filed a request to DEP for Noise
4 On 08/01/2018 12:01:00 AM , a user filed a request to DSNY for Request Large Bulky Item Collection


In [61]:
# note: you still need to use df['column_name'] when trying to select multiple columns from a dataframe

# this works using a single column
df.agency[df.incident_zip == '10012']

357         TLC
1340        DOT
2127        DEP
2417        DOB
2533       NYPD
           ... 
2538371    DSNY
2538573     DOB
2538724     DOT
2538843    NYPD
2539395    NYPD
Name: agency, Length: 3131, dtype: object

In [62]:
# if you want more columns, you have to provide them as a list
df[['agency','incident_zip']][df.incident_zip == '10012']

,agency,incident_zip
357,TLC,10012
1340,DOT,10012
2127,DEP,10012
2417,DOB,10012
2533,NYPD,10012
...,...,...
2538371,DSNY,10012
2538573,DOB,10012
2538724,DOT,10012
2538843,NYPD,10012


## Done with clean up! Time for the actual analysis: 
### Which 311 complaints are most common and which agencies are responsible for handling them?

Which complaints are the most common?

In [68]:
df.groupby('complaint_type').size().nlargest(15).reset_index(name='count')

# .reset_index(name='count') isn't necessary but it's helpful to include because it allows us to name the new column that contains the count of rows

,complaint_type,count
0,Noise - Residential,236350
1,HEAT/HOT WATER,222722
2,Illegal Parking,195159
3,Request Large Bulky Item Collection,177175
4,Blocked Driveway,145446
5,Street Condition,97178
6,Noise - Street/Sidewalk,95977
7,UNSANITARY CONDITION,85904
8,Street Light Condition,77755
9,Water System,74139


Which agencies are responsible for handling these complaint categories?

In [69]:
df.groupby(['agency', 'complaint_type']).size().nlargest(15).reset_index(name='count')

,agency,complaint_type,count
0,NYPD,Noise - Residential,236350
1,HPD,HEAT/HOT WATER,222722
2,NYPD,Illegal Parking,195159
3,DSNY,Request Large Bulky Item Collection,177175
4,NYPD,Blocked Driveway,145446
5,DOT,Street Condition,97178
6,NYPD,Noise - Street/Sidewalk,95977
7,HPD,UNSANITARY CONDITION,85904
8,DOT,Street Light Condition,77755
9,DEP,Water System,74139


Which agencies receive the most total 311 requests?

In [70]:
df.groupby('agency').size().nlargest(15).reset_index(name='count')

,agency,count
0,NYPD,850077
1,HPD,603043
2,DSNY,420165
3,DOT,298879
4,DEP,207280
5,DOB,149656
6,DPR,119970
7,DOHMH,72253
8,DOF,41436
9,TLC,35730


What is the most frequent request per agency?

In [74]:
# first create a dataframe that contains the count of complaints per agency per comaplaint typw
agency_counts = df_cleaned.groupby(['agency', 'complaint_type']).size().reset_index(name='count')

# use "drop_duplicates" function to keep the row with the highest value per agency
agency_counts.sort_values('agency').drop_duplicates('agency')

,agency,complaint_type,count
0,ACS,Forms,364
1,COIB,Forms,7
2,DCA,Consumer Complaint,16385
4,DCAS,Comments,68
6,DCP,Research Questions,22
18,DEP,Water System,74139
26,DFTA,Housing - Low Income Senior,4375
30,DHS,Homeless Person Assistance,20991
45,DOB,Investigations and Discipline (IAD),841
55,DOE,School Maintenance,2372


## Saving our cleaned dataframe as a csv on Google Drive

In the lefthand sidebar, click the `Files` icon, then `Mount Drive`.

In [75]:
df.to_csv('drive/My Drive/cleaned_311_data.csv')

# HOMEWORK 1

## Coding: Exploring Trends in How People Submit 311 Requests

Create a new Google Colab notebook called "HW1". Use the `cleaned_311_data.csv` file created during our lecture to complete the homework; it's shared with you on Google Drive.

```python
gdown.cached_download("https://drive.google.com/uc?id=1NBatP2xQbjH6Dd9mL-I8JMAISmeoMDht", "cleaned_311_data.csv")
df = pd.read_csv('cleaned_311_data.csv')
```

Answer these questions:

- Q1. Which complaint submission method (`Open Data Channel Type`) is used most frequently, according to this dataset? Provide the count of each for comparison.
- Q2. Get the count of each submission method per complaint type, sorted by complaint type.
    - Review the results and share a sentence or two with your observations.
    - Can you provide a potential explaination for why some complaints are more frequently made via phone and mobile versus on the website?
- Q3. Provide a simplified table that only lists the most common request submission method for each complaint type. There should be one row per complaint type, capturing the most common submission method for that complaint type.

then [turn in the assignment](https://github.com/afeld/python-public-policy#turning-in-assignments).

## Tutorials: Intro to Functions + Python Coding Best Practices

1. [Video introducing functions](https://www.youtube.com/watch?v=9Os0o3wzS_I&list=PL-osiE80TeTskrapNbzXhwoFUiLCjGgY7&index=8)
1. Coding Style Guides - Please skim these; I don't expect you to understand and follow everything in them. The most important guidelines to pay attention to are indentation and keeping each statement on its own line.
   1. https://docs.python-guide.org/writing/style/
   1. [PEP 8 Style Guide](https://www.python.org/dev/peps/pep-0008/)
1. [Guide to commenting your code](https://realpython.com/python-comments-guide/) 